# Sensitivity analysis of the NF-kB pathway
The steps for this are as follow:
1. From an equations text file, load the reaction equations and create a state space model
2. Derivate the sensitivity matrix (sbar) from the state space model
3. From the state space model and initial parameters, find 
    1. The initial parameters for the model (run it 1000 times from the first initial conditions)
    2. Determine the value of x<sub>i</sub> for all i for all t in [0, N]
4. Having the sensitivity matrix and the values of x for all desired t, calculate the sensitivity equations

### All necessary imports

In [2]:
import numpy as np
import sympy as sp
from scipy.integrate import odeint

### All program constants

In [20]:
initial_conditions_N = 2000
N = 400

pathway_file = 'pathway_eqs.txt'
reaction_coefficients_file = 'reaction_coefficients.txt'

## 1. Generate the state space model from reaction equations

In [48]:
import numpy as np

file = open(pathway_file,"r")
eqs_arr=file.readlines()
file.close()
eqs_arr=[i[:-1] if i[-1] == '\n' else i for i in eqs_arr ] #remove \n

class Rxn:
    
    def __init__(self, ID, reactants, products):
        #reactants and products are lists
        self.ID = str(ID)
        self.reactants = reactants
        self.products = products
        
    def __str__(self):
        ret = "ID: " + self.ID + " Reactants: "
        for r in self.reactants:
            ret = ret + r.ID + ", "
        ret = ret + "Products: "
        for p in self.products:
            ret = ret + p.ID + ", "
        return ret

class RxnSpec:
    
    def __init__(self, ID, name):
        self.ID = 'x' + str(ID)
        self.name = name
        
    def __str__(self):
        return 'ID: ' + self.ID + ' name: ' + self.name

#list of rxn species
rxn_species = [RxnSpec(1,'ikba'), RxnSpec(2,'nf-kb'), RxnSpec(3,'ikba-nf-kb'), RxnSpec(4,'ikbb'), RxnSpec(5,'ikbb-nf-kb'),
              RxnSpec(6,'ikbe'),RxnSpec(7,'ikbe-nf-kb'),RxnSpec(8,'ikkikba'),RxnSpec(9,'ikkikba-nf-kb'),RxnSpec(10,'ikk'),
              RxnSpec(11,'ikkikbb'),RxnSpec(12,'ikkikbb-nf-kb'),RxnSpec(13,'ikkikbe'),RxnSpec(14,'ikkikbe-nf-kb'),
              RxnSpec(15,'nf-kb_n'),RxnSpec(16,'ikba_n'),RxnSpec(17,'ikba_n-nf-kb_n'),RxnSpec(18,'ikbb_n'),RxnSpec(19,'ikbb_n-nf-kb_n'),
              RxnSpec(20,'ikbe_n'),RxnSpec(21,'ikbe_n-nf-kb_n'),RxnSpec(22,'ikba_-t'),RxnSpec(23,'ikbb_-t'),RxnSpec(24,'ikbe_-t'),RxnSpec('SOURCE','source'),RxnSpec('SINK','sink')]

rxn_arr = []

for idx,eq in enumerate(eqs_arr):
    temp = eq.split('->')
    reactants = temp[0].split('+')
    products = temp[1].split('+')
    #convert each element in reactants and products into a RxnSpec
    new_reactants = []
    new_products = []
    for reac in reactants:
        for rs in rxn_species:
            if rs.name == reac:
                new_reactants.append(rs)
                break
    for prod in products:
        for rs in rxn_species:
            
            if rs.name == prod:
                new_products.append(rs)
                break
    
    
    rxn = Rxn(idx+1,new_reactants, new_products)
    rxn_arr.append(rxn)


for rxn in rxn_arr:
    print(rxn)

state_space_model = []

for rs in rxn_species:
    eq = rs.ID + '(t) = '
    for rxn in rxn_arr:
        if rs.name in [i.name for i in rxn.reactants]:
            k = '-k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
            
        if rs.name in [i.name for i in rxn.products]:
            k = '+k' + str(rxn.ID)
            eq = eq + k
            for reac in rxn.reactants:
                eq = eq + '*' + reac.ID
    state_space_model.append(eq)
    
for eq in state_space_model:
    print(eq)

ID: 1 Reactants: x1, x2, Products: x3, 
ID: 2 Reactants: x3, Products: x2, x1, 
ID: 3 Reactants: x4, x2, Products: x5, 
ID: 4 Reactants: x5, Products: x2, x4, 
ID: 5 Reactants: x6, x2, Products: x7, 
ID: 6 Reactants: x7, Products: x2, x6, 
ID: 7 Reactants: x8, x2, Products: x9, 
ID: 8 Reactants: x9, Products: x2, x8, 
ID: 9 Reactants: x9, Products: x10, x2, 
ID: 10 Reactants: x11, x2, Products: x12, 
ID: 11 Reactants: x12, Products: x2, x11, 
ID: 12 Reactants: x12, Products: x10, x2, 
ID: 13 Reactants: x13, x2, Products: x14, 
ID: 14 Reactants: x14, Products: x2, x13, 
ID: 15 Reactants: x14, Products: x10, x2, 
ID: 16 Reactants: x3, Products: x2, 
ID: 17 Reactants: x5, Products: x2, 
ID: 18 Reactants: x7, Products: x2, 
ID: 19 Reactants: x2, Products: x15, 
ID: 20 Reactants: x15, Products: x2, 
ID: 21 Reactants: x16, x15, Products: x17, 
ID: 22 Reactants: x17, Products: x15, x16, 
ID: 23 Reactants: x18, x15, Products: x19, 
ID: 24 Reactants: x19, Products: x15, x18, 
ID: 25 Reactants: 

## 2. Derivate the sensitivity matrix (sbar)

### s̅ derivation
 s̅<sub>i,j</sub> is the value needed for calculating sensitivities
 The formula for it is the following **(from equation 13 in the Yue et. al. paper)**
 
 ```
  s̅_i,j = (dx_i)/(dtheta_j) * theta_j/x_i 
 ```
 
 where `i` is the i<sup>th</sup> state variable and `j` is j<sup>th</sup> parameter value
 
 
 x<sub>i</sub> is the i<sup>th</sup> entry in the state matrix X which is defined as
 ```
 X = [x1, x2, ...]
 ```
 
 theta<sub>j</sub> is the j<sup>th</sup> entry in the theta matrix which is defined as 
 ```
 theta = [k1, k2, ...]
 ```
 

In [65]:
eqts = []
xs = []
thetas = []

theta_char = 'k'
extras = ['(', ')', '[', ']']

def get_symbols(string):
    symbols = []
    nopows = string.split('**')
    for nopow in nopows:
        nodivs = nopow.split('/')
        for nodiv in nodivs:
            nomults = nodiv.split('*')
            for nomult in nomults:
                noadds = nomult.split('+')
                for noadd in noadds:
                    nosub = noadd.split('-')
                    for s in nosub:
                        for ext in extras:
                            s = s.replace(ext, '')
                        symbols.append(s)
    return symbols


for eq in state_space_model:
    x = eq.split('=')[0].split('(')[0]
    if x not in xs:
        xs.append(x)
    eqt = eq.split('=')[1].strip()
    symbols = get_symbols(eqt)
    for s in symbols:
        if theta_char in s and s not in thetas:
            thetas.append(s)

    eqts.append((x, eqt))
    
thetas.sort(key=lambda x: int(x[1:]))   
xs = xs[:-2]
print(xs)
print(thetas)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24']
['k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11', 'k12', 'k13', 'k14', 'k15', 'k16', 'k17', 'k18', 'k19', 'k20', 'k21', 'k22', 'k23', 'k24', 'k25', 'k26', 'k27', 'k28', 'k29', 'k30', 'k31', 'k32', 'k33', 'k34', 'k35', 'k36', 'k37', 'k38', 'k39', 'k40', 'k41', 'k42', 'k43', 'k44', 'k45', 'k46', 'k47', 'k48', 'k49', 'k50', 'k51', 'k52', 'k53', 'k54', 'k55', 'k56', 'k57', 'k58', 'k59', 'k60', 'k61', 'k62', 'k63', 'k64']


### Find the derivatives
The derivative list will by `i` by `j` in size, with `i` being the number of `x` variables and `j` being the number of `theta` variables

In [66]:
derivatives = [['' for j in range(len(thetas))] for i in range(len(xs))]
for i in range(len(xs)):
    for j in range(len(thetas)):
        derivatives[i][j] = sp.diff(eqts[i][1], thetas[j])

### Calculate s̅

In [77]:
s_bar = [['' for j in range(len(thetas))] for i in range(len(xs))]
for i in range(len(xs)):
    for j in range(len(thetas)):
        s_bar[i][j] = sp.Mul((sp.Mul(sp.sympify(derivatives[i][j]) * sp.sympify(thetas[j]))), sp.sympify('{}**-1'.format(xs[i])))
        
print(s_bar)

[[-2.180038243575744*k1*x1*x2, 2.180038243575744*k2*x3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.180038243575744*k34*x1*x10, 2.180038243575744*k35*x8, 2.180038243575744*k36*x22, -2.180038243575744*k37*x1, -2.180038243575744*k38*x1, 2.180038243575744*k39*x16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-849.20281139417791*k1*x1*x2, 849.20281139417791*k2*x3, -849.20281139417791*k3*x2*x4, 849.20281139417791*k4*x5, -849.20281139417791*k5*x2*x6, 849.20281139417791*k6*x7, -849.20281139417791*k7*x2*x8, 849.20281139417791*k8*x9, 849.20281139417791*k9*x9, -849.20281139417791*k10*x11*x2, 849.20281139417791*k11*x12, 849.20281139417791*k12*x12, -849.20281139417791*k13*x13*x2, 849.20281139417791*k14*x14, 849.20281139417791*k15*x14, 849.20281139417791*k16*x3, 849.20281139417791*k17*x5, 849.20281139417791*k18*x7, -849.20281139417791*k19*x2, 849.20281139417791*k20*x15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## 3. Identify the initial conditions and x(t) for all t
1. Run the integration initial_conditions_N number of times to get initial conditions
2. Augment these by any thing you need from the paper for actual initial conditions

__NOTE__:   as of now, hard code the k values (from some source) and the partial derivatives (from the state space model)

In [69]:
# followed the direction of this video https://www.youtube.com/watch?v=zRMmiBMjP9o
def nf_kb(x, t):
    # x is here to be used by (1) initial conditions and (2) x when we use odeint
    # t is here for obvious reasons
    
    # define the k values
    k1=0.5
    k2=0.5*10**-3 
    k3=0.5
    k4=0.5*10**-3 
    k5=0.5
    k6=0.5*10**-3 
    k7=0.5
    k8=0.5*10**-3 
    k9=2.04*10**-2 
    k10=0.5
    k11=0.5*10**-3 
    k12=7.5*10**-3 
    k13=0.5
    k14=0.5*10**-3 
    k15=1.1*10**-2 
    k16=2.25*10**-5 
    k17=2.25*10**-5 
    k18=2.25*10**-5 
    k19=0.9*10**-1 
    k20=0.8*10**-4 
    k21=0.5
    k22=0.5*10**-3 
    k23=0.5
    k24=0.5*10**-3 
    k25=0.5
    k26=0.5*10**-3 
    k27=1.54*10**-6  
    k28=1.65*10**-2  
    k29=2.8*10**-4 
    k30=1.78*10**-7  
    k31=2.8*10**-4 
    k32=1.27*10**-7  
    k33=2.8*10**-4 
    k34=22.5*10**-3  
    k35=1.25*10**-3 
    k36=4.08*10**-3 
    k37=1.13*10**-4 
    k38=3*10**-4 
    k39=2*10**-4 
    k40=6.0*10**-3  
    k41=1.75*10**-3 
    k42=4.08*10**-3 
    k43=1.13*10**-4 
    k44=1.5*10**-4 
    k45=1*10**-4 
    k46=9.0*10**-3  
    k47=1.75*10**-3 
    k48=4.08*10**-3 
    k49=1.13*10**-4 
    k50=1.5*10**-4 
    k51=1*10**-4 
    k52=1.85*10**-1  
    k53=1.25*10**-3 
    k54=1.38*10**-2 
    k55=4.8*10**-2  
    k56=1.75*10**-3 
    k57=5.2*10**-3 
    k58=7.0*10**-2  
    k59=1.75*10**-3 
    k60=5.2*10**-3 
    k61=1.2*10**-4 
    k62=4.07*10**-3 
    k63=1.5*10**-3 
    k64=2.2*10**-3 
    
    # im lazy and make it easy to read
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    x4 = x[3]
    x5 = x[4]
    x6 = x[5]
    x7 = x[6]
    x8 = x[7]
    x9 = x[8]
    x10 = x[9]
    x11 = x[10]
    x12 = x[11]
    x13 = x[12]
    x14 = x[13]
    x15 = x[14]
    x16 = x[15]
    x17 = x[16]
    x18 = x[17]
    x19 = x[18]
    x20 = x[19]
    x21 = x[20]
    x22 = x[21]
    x23 = x[22]
    x24 = x[23]
    
    dx1dt = -k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16
    dx2dt = -k1*x1*x2+k2*x3-k3*x4*x2+k4*x5-k5*x6*x2+k6*x7-k7*x8*x2+k8*x9+k9*x9-k10*x11*x2+k11*x12+k12*x12-k13*x13*x2+k14*x14+k15*x14+k16*x3+k17*x5+k18*x7-k19*x2+k20*x15
    dx3dt = +k1*x1*x2-k2*x3-k16*x3-k52*x10*x3+k53*x9+k54*x17
    dx4dt = -k3*x4*x2+k4*x5-k40*x10*x4+k41*x11+k42*x23-k43*x4-k44*x4+k45*x18
    dx5dt = +k3*x4*x2-k4*x5-k17*x5-k55*x10*x5+k56*x12+k57*x19
    dx6dt = -k5*x6*x2+k6*x7-k46*x10*x6+k47*x13+k48*x24-k49*x6-k50*x6+k51*x20
    dx7dt = +k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21
    dx8dt = -k7*x8*x2+k8*x9+k34*x10*x1-k35*x8-k62*x8
    dx9dt = +k7*x8*x2-k8*x9-k9*x9+k52*x10*x3-k53*x9
    dx10dt = +k9*x9+k12*x12+k15*x14-k34*x10*x1+k35*x8-k40*x10*x4+k41*x11-k46*x10*x6+k47*x13-k52*x10*x3+k53*x9-k55*x10*x5+k56*x12-k58*x10*x7+k59*x14-k61*x10+k62*x8+k63*x11+k64*x13
    dx11dt = -k10*x11*x2+k11*x12+k40*x10*x4-k41*x11-k63*x11
    dx12dt = +k10*x11*x2-k11*x12-k12*x12+k55*x10*x5-k56*x12
    dx13dt = -k13*x13*x2+k14*x14+k46*x10*x6-k47*x13-k64*x13
    dx14dt = +k13*x13*x2-k14*x14-k15*x14+k58*x10*x7-k59*x14
    dx15dt = +k19*x2-k20*x15-k21*x16*x15+k22*x17-k23*x18*x15+k24*x19-k25*x20*x15+k26*x21-k28*x15*x15+k28*x15*x15
    dx16dt = -k21*x16*x15+k22*x17+k38*x1-k39*x16
    dx17dt = +k21*x16*x15-k22*x17-k54*x17
    dx18dt = -k23*x18*x15+k24*x19+k44*x4-k45*x18
    dx19dt = +k23*x18*x15-k24*x19-k57*x19
    dx20dt = -k25*x20*x15+k26*x21+k50*x6-k51*x20
    dx21dt = +k25*x20*x15-k26*x21-k60*x21
    dx22dt = +k27+k28*x15*x15-k29*x22-k36*x22+k36*x22
    dx23dt = +k30-k31*x23-k42*x23+k42*x23
    dx24dt = +k32-k33*x24-k48*x24+k48*x24
    
    
    return [dx1dt, dx2dt, dx3dt, dx4dt, dx5dt, dx6dt, dx7dt, dx8dt,
            dx9dt, dx10dt, dx11dt, dx12dt, dx13dt, dx14dt, dx15dt, dx16dt, 
            dx17dt, dx18dt, dx19dt, dx20dt, dx21dt, dx22dt, dx23dt, dx24dt]
# first run for the initial conditions
T = initial_conditions_N
x0x0 = [0, 0.1] + [0 for i in range(22)]
t = np.asarray(range(T))

init_conditions = odeint(nf_kb, x0x0, t)

Now we have the initial conditions, so we need to augment this by paper specific parameters

In [70]:
# ok now do it for the simulation
T = N
x0 = init_conditions[-1]
# fill in the extra stuff in the paper, IKK at 0.1 (x10, index 9) PAPER SPECIFIC
x0[9] = 0.1
t = np.asarray(range(T))

x = odeint(nf_kb, x0, t)
# x has the shape of 
# [N, i] (an entry for all x for all N)

## 4 the sensitivity constants 

### Dynamic sensitivity analysis
ok so we have these components
* x<sub>i</sub>(t) for all i in [1, 24] (in `x`)
* sensitivity matrix (sbar) as a function of k<sub>j</sub> and x<sub>i</sub> (in `s_bar`)
* k<sub>j</sub> for all j [1, 64] (in `reaction_coefficients.txt`)

Using these components, we can calculate equations (21) and (22)

In [71]:
# get x(t) for all t
xt = {}
for xs in x:
    for i in range(len(xs)):
        key = 'x{}'.format(i+1) 
        if key not in xt:
            xt[key] = []
        xt[key].append(xs[i])
        
# get all of the ks
ks = {}
with open('./reaction_coefficients.txt', 'r') as o:
    for line in o:
        k = line.split('=')[0]
        ks[k] = sp.sympify(line.split('=')[1].replace('\n', '').strip()).evalf()
        

#### Dynamic sensitivity analysis with multiple variables
This is equation 22 of the paper
![Equation 22](./equation22.png)

In [78]:
# calc sbar 
def sbar(i, j, k):
    # get all the substitute values for the expression
    # get the row of xt for the k
    tosub = {}
    for key in xt.keys():
        tosub[key] = xt[key][k]
    # get all the ks
    for key in ks.keys():
        tosub[key] = ks[key]
    # add xSOURCE here becuase i'm lazy
    tosub['xSOURCE'] = 1
    # get the i j entry of sbar
    s = s_bar[i][j] 
    # substitute and eval
    a = s.evalf(subs=tosub)
    b = float(a)
    return b

In [79]:
# do the intermost summation
def inner_sum(k, j, i_range):
    tosum = []
    for i in range(i_range):
        tosum.append(sbar(i, j, k) ** 2)
    return sum(tosum)

In [80]:
# do the outer summation
def outer_sum(N, j, i_range):
    tosum = []
    for k in range(N):
        tosum.append(inner_sum(k, j, i_range))
    return sum(tosum)

In [81]:
# do the 1/n and the sqrt
def eqn22(N, j, i_range):
    return (1/N) * np.sqrt(outer_sum(N, j, i_range))

In [82]:
# do the sensitivity analysis
i_range = len(xt.keys())
j_range = len(ks.keys())
OS = []
for j in range(j_range):
    print('Calculating sensitivity for j {}/{}\r'.format(j, j_range), end='')
    OS.append(eqn22(N, j, i_range))
    

In [60]:
print(OS)

[0.012606868477709383, 0.002139742008698113, 3.0284490420657284e-05, 2.50000000000001e-05, 3.0284490420657284e-05, 2.50000000000001e-05, 0.00117954224835337, 0.0004040432246874984, 0.016488556023871757, 3.0284490420657284e-05, 2.50000000000001e-05, 0.0003750000000000015, 3.0284490420657284e-05, 2.50000000000001e-05, 0.0005500000000000021, 9.628826272690887e-05, 1.1249999999999985e-06, 1.1249999999999985e-06, 0.004792544205984789, 1.2788883107641217e-05, 0.0017064429323142526, 5.232680969201812e-05, 9.127470937396257e-05, 2.50000000000001e-05, 9.127470937396257e-05, 2.50000000000001e-05, 7.699999999999967e-08, 1.0584138100139348e-07, 1.8254311526631807e-05, 8.9e-09, 1.3999999999999985e-05, 6.350000000000014e-09, 1.3999999999999985e-05, 0.0031594224543595247, 0.00011729860416039133, 4.441714826725462e-05, 2.2699662897383745e-05, 0.00013393158061274723, 1.0068258476612847e-05, 1.0843891228224274e-05, 8.750000000000031e-05, 0.0, 5.650000000000005e-06, 7.500000000000027e-06, 4.9999999999999

In [61]:
np.argmax(OS)

8

In [62]:
counted = [(i, OS[i]) for i in range(len(OS))]

In [63]:
counted.sort(key=lambda x: x[1], reverse=True)

In [64]:
print(counted)

[(8, 0.016488556023871757), (0, 0.012606868477709383), (18, 0.004792544205984789), (51, 0.0033206640018099086), (33, 0.0031594224543595247), (1, 0.002139742008698113), (20, 0.0017064429323142526), (6, 0.00117954224835337), (53, 0.0006931563676635437), (14, 0.0005500000000000021), (7, 0.0004040432246874984), (61, 0.0003814210718457893), (11, 0.0003750000000000015), (56, 0.00026000000000000074), (59, 0.00026000000000000074), (37, 0.00013393158061274723), (57, 0.00012651206432928326), (34, 0.00011729860416039133), (63, 0.00011000000000000048), (15, 9.628826272690887e-05), (22, 9.127470937396257e-05), (24, 9.127470937396257e-05), (40, 8.750000000000031e-05), (46, 8.750000000000031e-05), (55, 8.750000000000031e-05), (58, 8.750000000000031e-05), (54, 8.67511298257942e-05), (52, 7.558314271049252e-05), (62, 7.499999999999968e-05), (21, 5.232680969201812e-05), (35, 4.441714826725462e-05), (2, 3.0284490420657284e-05), (4, 3.0284490420657284e-05), (9, 3.0284490420657284e-05), (12, 3.028449042065